# Calculations - degrader foil thicknesses

In [ ]:
%run SRIM_Ca48.ipynb
%run SRIM_U225.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg
from matplotlib import colors
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'];
#Customising style
#mpl.rc('lines', linewidth=2, color='r')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
#mpl.rcParams['savefig.format'] = "eps"
#mpl.rcParams['savefig.dpi'] = 100
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['figure.figsize'] = (16,9)

## Initialise general parameters

In [ ]:
e_tank1 = 4.774 #MeV/u
e_tank2 = 5.893 #MeV/u
A_beam = 48

## Hf for shortlived alpha-decaying nuclei

Ca-48 on Hf-nat.

Beam energies at middle of target should be aimed for: 

* 4.2 MeV/u

This is according to Andrew's calculations (got them from Dirk)

In [ ]:
e_tank = e_tank1
target = "Hf"
degrader = "Al"
backing = "Ti"

In [ ]:
degrader_thickness = 0.76 # mg/cm2
degrader_thickness = 1e4*(degrader_thickness / 2.70e3) #µm
print(degrader_thickness)
degrader_thickness = 0*2 #µm

backing_thickness = 0.82 # mg/cm2 (according to Dirk)
backing_thickness = 1e4*(backing_thickness/4.506e3) # µm
backing_thickness = 2.25 # µm

target_thickness = 0.83 #mg/cm2 (correspondence with Bettina Lommel)
target_thickness = 1e4*(target_thickness/13.31e3) # µm

### Traversing material and calculating energy

In [ ]:
E_of_X = [] #energies
X = [] # distance
E = e_tank*A_beam
E_of_X.append(E)
X.append(0)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# degrader
E = E - e_loss_x(E, degrader_thickness, degrader)
E_of_X.append(E)
X.append(X[-1]+degrader_thickness)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# backing
E = E - e_loss_x(E, backing_thickness, backing)
E_of_X.append(E)
X.append(X[-1]+backing_thickness)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# target, middle and end
E = E - e_loss_x(E, target_thickness*0.5, target)
E_of_X.append(E)
X.append(X[-1]+target_thickness*0.5)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')
E = E - e_loss_x(E, target_thickness*0.5, target)
E_of_X.append(E)
X.append(X[-1]+target_thickness*0.5)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

E_of_X = np.asarray(E_of_X)
X = np.asarray(X)

In [ ]:
stop_labels = ['start_degrader', 'start_backing', 'start_target', 'middle_target', 'end_target']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,10))
axes[0].plot(X, E_of_X, label='E(X)')
axes[0].set_xlabel("Traversed material (µm)")
axes[0].set_ylabel('Energy (MeV))')
axes[1].plot(stop_labels, E_of_X/A, label='E(X)')
plt.xticks(rotation=90)
axes[1].set_xlabel("Traversed material")
axes[1].set_ylabel('Energy (MeV/u)')
plt.show()

# Fusion: U-228 excitation and kinetic energies

In [ ]:
from pyne import data as pyne_data

In [ ]:
A_compound = 228
masses = {"Ca48": pyne_data.atomic_mass("Ca48"), "Hf180": pyne_data.atomic_mass("Hf180"), "U228": pyne_data.atomic_mass("U228")}
for m in masses:
    masses[m] *= 931.494 #MeV/c^2
masses

Looping target

In [ ]:
ind_start = 2
E_compound, E_exc = [], []

for i in range(ind_start, len(E_of_X)):
    E_beam = E_of_X[i]
    E_kin_compound =  E_beam * (A_beam/A_compound)
    excitation_energy = masses["Ca48"] + masses["Hf180"] - masses["U228"] + E_beam - E_kin_compound    
    E_compound.append(E_kin_compound)
    E_exc.append(excitation_energy)

print("Fusion at: ", stop_labels[ind_start:])
print("Compound kinetic energy: ", E_compound)
print("Compound excitation energy: ", E_exc)